# Scraping WodUp

In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import sys
sys.path.append('../src/')
import wodupcrawler
import os
from bokeh.models import Div
from bokeh.io import output_notebook, show
import fitetl, wodupcrawler
import glob
import pandas as pd
from datetime import datetime

output_notebook()

Loading BokehJS ...

In [9]:
# Heart rate .fit files
#fitetl.main()
df_hr_rcvry = fitetl.read_rcvry_csv(fitetl.datadir_hrsum)
df_hr_profile = fitetl.read_hr_profile_csv(fitetl.datadir_hrts)
ts_files = sorted(glob.glob(f'{fitetl.datadir_hrts}*.csv'))

# WOD logs
#wodupcrawler.main()
wods, df_wods = wodupcrawler.read_wods_json(f'{wodupcrawler.datadir}session_wods.json')
latest_wodup_log_dt = wodupcrawler.get_latest_wodup_log_date(wods)

[########################################] | 100% Completed |  1.1s


In [46]:
df_cal = pd.DataFrame({'date': pd.date_range('2019-09-01', datetime.today())})
df_cal = df_cal.merge(df_hr_rcvry, on='date', how='left').merge(df_wods, on='date', how='left')
df_cal = df_cal[['date','calories', 'max_hr', '120_sec_rec', 'html', '174_220']]

In [48]:
missing_wahoo = df_cal.calories.isna() & (df_cal.html != "<p>&nbsp;</p><p>&nbsp;</p><p>&nbsp;</p>")
df_cal.loc[missing_wahoo,'calories'] = 300

In [57]:
df_hr_profile.head()

,s,2020-05-25,2020-05-26,2020-06-16,2020-06-19,2020-06-20,2020-06-23,2020-06-25,2020-06-30,2020-07-06,...,2020-10-28,2020-10-29,2020-10-30,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,Time,BPM
0,0,87.0,102.0,45.0,91.0,82.0,74.0,NaN,69.0,90.0,...,62.0,62.0,89.0,99.0,67.0,NaN,99.0,NaN,00:00:00,NaN
1,1,87.0,102.0,45.0,91.0,82.0,74.0,72.0,70.0,89.0,...,63.0,63.0,88.0,91.0,67.0,59.0,98.0,86.0,00:00:01,86.0
2,2,87.0,102.0,45.0,91.0,82.0,74.0,72.0,72.0,89.0,...,63.0,63.0,86.0,85.0,67.0,59.0,97.0,86.0,00:00:02,86.0
3,3,96.0,103.0,45.0,91.0,74.0,101.0,72.0,73.0,89.0,...,63.0,63.0,84.0,84.0,66.0,59.0,97.0,87.0,00:00:03,87.0
4,4,133.0,111.0,45.0,83.0,74.0,104.0,72.0,74.0,86.0,...,62.0,62.0,83.0,84.0,65.0,59.0,97.0,88.0,00:00:04,88.0


In [59]:
df_wods.min()

date                                  2019-09-16 00:00:00
html    <div class="flex items-center w-primary mb2 f5...
dtype: object

In [75]:
show(Div(text="""
<h2>Monday</h2>
    <ul>
        <li>Back Squat: 5X10 @ 225 lbs</li>
        <li>Bench Press: 5X5 @ 195 lbs</li>
        <li>AMRAP 15 minutes</li>
            <ul>
                <li>5 Deadlifts</li>
                <li>5 Pendalay rows</li>
                <li>5 Power cleans</li>
                <li>5 Push jerks</li>
                <li>5 Front squats</li>
            </ul>
    </ul>
<h2>Tuesday</h2>
    <ul>
        <li>Ring muscle ups 10x5</li>
        <li>Handstand push ups 5x5</li>
        <li>Box pistols 5x10</li>
        <li>Run: 5-10km</li>
    </ul>
<h2>Wednesday</h2>
    <ul>
        <li>Deadlift: 5X5 @ 305 lbs</li>
        <li>Bench Press: 4X4 @ 195 lbs</li>
        <li>4 Rounds for time:</li>
            <ul>
                <li>4 Strict pull ups</li>
                <li>8 Burpees</li>
                <li>16 Box jumps</li>
                <li>32 Double Unders</li>
            </ul>
    </ul>
<h2>Thursday</h2>
    <ul>
        <li>Ring muscle ups 10x5</li>
        <li>Handstand push ups 5x5</li>
        <li>Box pistols 5x10</li>
        <li>Run: 5-10km</li>
    </ul>
<h2>Friday</h2>
    <ul>
        <li>Deadlift: 4X4 @ 315 lbs</li>
        <li>Banded Bench press: 5X5 @ 165 lbs</li>
        <li>4 Rounds for time:</li>
            <ul>
                <li>4 Deficit handstand push ups</li>
                <li>8 Ring dips</li>
                <li>16 Box jumps overs</li>
                <li>32 Calories of assault bike</li>
            </ul>
    </ul>
"""))